For questions about using the API, see https://quickstats.nass.usda.gov/api

In [ ]:
import json
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
def get_data(years, states='all'):
    """
    Retrieves data from NASS USDA Quick Stats API. 
    
    Feel free to change what is in the config_url to pull different data, 
    and feel free to change the columns that are displayed in the output df.
    Available features are listed in the docs above.
    NOTE: If a requested state has no data for a given year, it is simply skipped.
    
    Args:
        years (list): List of years to retrive. 
        states (list or str): List of states to retrieve, or 'all'
    
    Returns:
        (DataFrame): dataframe concatenated by state then year
    """
    api_key = '986A28EC-EF35-31C1-A179-461CF9DF4F8E'
    root_url = 'http://quickstats.nass.usda.gov/api/api_GET/?key={}'.format(api_key)
    
    if states == 'all':
        states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
                  "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
                  "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
                  "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
                  "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
        
    all_dfs = []
    with tqdm(total=len(years)*len(states)) as pbar:
        for year in years:
            for state in states:
                # Parameters
                source_desc = 'SURVEY' # Program
                sector_desc = 'CROPS' # Sector
                group_desc = 'FIELD CROPS' # Group
                agg_level_desc = 'COUNTY' # Geographical Level
                year = year # Year
                freq_desc = 'ANNUAL' # Period Type
                reference_period_desc = 'YEAR' # Period
                statisticcat_desc = 'AREA HARVESTED' # Data Item
                state_alpha = state

                params = [source_desc, sector_desc, group_desc, agg_level_desc,
                          year, freq_desc, reference_period_desc, statisticcat_desc,
                          state_alpha]

                config_url = '&source_desc={}'\
                             '&sector_desc={}'\
                             '&group_desc={}'\
                             '&agg_level_desc={}'\
                             '&year={}'\
                             '&freq_desc={}'\
                             '&reference_period_desc={}'\
                             '&statisticcat_desc={}'\
                             '&state_alpha={}'.format(*params)

                response = requests.get(root_url + config_url)
                if response.status_code == 200:
                    data = json.loads(response.content.decode('utf-8'))
                    df = pd.DataFrame(data['data'])

                    ###########################################################################
                    # If you want to change the output dataframe, change this list of columns. 
                    # Possible columns can be found at the above URL under "Usage"
                    ###########################################################################
                    df['fips_code'] = df['state_fips_code'] + df['county_code'] # Note: This might not actually be a valid FIPS code in some cases
                    columns = ['year', 'fips_code', 'state_alpha', 
                               'county_name', 'commodity_desc', 'unit_desc', 'Value','short_desc']
                    df = df[columns]
                    df = df.rename(columns={'Value':'value'})
                    df = df.sort_values(by=['fips_code', 'commodity_desc'])

                    all_dfs.append(df)
                pbar.update()
                
    return pd.concat(all_dfs, ignore_index=True)

df = get_data(years=np.arange(1900, 2021), states=['ND'])

100%|██████████| 21/21 [00:56<00:00,  2.68s/it]


In [ ]:
df.head()

,year,fips_code,state_alpha,county_name,commodity_desc,unit_desc,value
0,2000,38001,ND,ADAMS,BARLEY,ACRES,"8,200"
1,2000,38001,ND,ADAMS,CANOLA,ACRES,"2,300"
2,2000,38001,ND,ADAMS,CORN,ACRES,"4,000"
3,2000,38001,ND,ADAMS,CORN,ACRES,"4,900"
4,2000,38001,ND,ADAMS,CORN,ACRES,"4,000"


In [ ]:
df[df.value == '                 (D)'] = np.NaN
df.dropna()
df['value'] = df['value'].str.replace(',','')
print(df.tail())
df['value'] = df['value'].astype('float64')
df.head()

         year fips_code state_alpha     county_name commodity_desc unit_desc  \
14153  2020.0     38998          ND  OTHER COUNTIES       SOYBEANS     ACRES   
14154  2020.0     38998          ND  OTHER COUNTIES      SUNFLOWER     ACRES   
14155  2020.0     38998          ND  OTHER COUNTIES          WHEAT     ACRES   
14156  2020.0     38998          ND  OTHER COUNTIES          WHEAT     ACRES   
14157  2020.0     38998          ND  OTHER COUNTIES          WHEAT     ACRES   

        value  
14153   44750  
14154  144900  
14155  171100  
14156  251080  
14157   33000  


,year,fips_code,state_alpha,county_name,commodity_desc,unit_desc,value
0,2000.0,38001,ND,ADAMS,BARLEY,ACRES,8200.0
1,2000.0,38001,ND,ADAMS,CANOLA,ACRES,2300.0
2,2000.0,38001,ND,ADAMS,CORN,ACRES,4000.0
3,2000.0,38001,ND,ADAMS,CORN,ACRES,4900.0
4,2000.0,38001,ND,ADAMS,CORN,ACRES,4000.0


In [ ]:
"""county_dict is a dictionary containing the name of the county as the key and a dataframe for the county as a value"""
county_dict = {}
for county in df.county_name.unique():
    data = df[df['county_name']==county]
    county_dict.update({county:data})

# The splitter returns a subset of the data based on years and crop type

In [ ]:
def splitter(county,year_max,year_min,crop,desc=None):
    sub = county_dict[county]
    if desc != None:
        crop = sub.loc[(sub['year']>=year_min)&(sub['year']<=year_max) & (sub['commodity_desc'] == crop) & (sub['short_desc'] == desc)]
    else:
        crop = sub.loc[(sub['year']>=year_min)&(sub['year']<=year_max) & (sub['commodity_desc'] == crop)]
    crop.set_index('year',inplace = True)
    return crop